In [1]:
from pathlib import Path
from chic import Structure, Net

---
## Coarse-graining supercells

In [2]:
# path to test example directory.
eg_dir = Path('cg_zif8')

# instantiate a Structure object from a CIF file.
zif8 = Structure.from_cif(eg_dir / 'ZIF-8-sod.cif', cores=4, verbose=True)
zif8.remove_species('O')
zif8.average_element_pairs('H', rmin=0.0, rmax=0.9)

average_element_pairs() took 0.02 seconds to execute.


/Users/tcnicholas/miniconda3/envs/chic23/lib/python3.8/site-packages/chic/cif.py:107: UserWarning: Unable to match CIF atoms to Pymatgen atoms.
  warnings.warn('Unable to match CIF atoms to Pymatgen atoms.')


In [3]:
# we *strongly* reccomend you first compute the neighbour list for your
# structure *before* you replicate it, otherwise you will have to recompute
# the neighbour list for all atoms in the supercell.
zif8.get_neighbours_crystalnn()

get_neighbours_crystalnn() took 8.75 seconds to execute.


In [4]:
# now we can replicate the cell. we pass it a list of three integers, which are 
# the number of times we want to replicate the unit cell in each direction. in 
#this case, we replicate the unit cell once in the x direction, three times in 
# the y direction, and twice in the z direction. it returns a new Net object, 
# which we can assign to a new variable.
zif8_supercell = zif8.replicate([1, 3, 2])

replicate() took 0.88 seconds to execute.


In [5]:
# now we can proceed as normal, since we have now got a supercell and the 
# neighbour list has been updated to account for it.
zif8_supercell.find_atomic_clusters()
zif8_supercell.get_coarse_grained_net()
zif8_supercell.net_to_cif(
    eg_dir / 'ZIF-8-sod-cg-1x3x2.cif', 
    write_bonds=True, 
    name='ZIF-8-cg-supercell'
)
zif8_supercell.net_to_lammps_data(
    eg_dir / 'ZIF-8-sod-cg-1x3x2.data',
    write_bonds=True,
    name='ZIF-8-cg-supercell'
)

find_atomic_clusters() took 0.20 seconds to execute.
get_coarse_grained_net() took 0.08 seconds to execute.


---
## Decorating supercells

You might have the TopoCIF for a single unit cell, but need a larger decorated 
net (e.g. for simulation). We can achieve this with the replicate() function.

One particular use-case might be when using the structure for LAMMPS
simulations, where explicit bonds are given. Sometimes the unit cells might be 
too small for the minimum image convention (MIC) to work, and therefore the 
force field will fail.

In [6]:
# path to test example directory.
eg_dir = Path('decorate_sod')

# instantiate a Net object from a topocif file.
sod_net = Net.from_topocif(eg_dir / 'ZIF-8-sod-cg.cif')

In [7]:
# now that we have our Net object, we can replicate the unit cell. we pass it
# a list of three integers, which are the number of times we want to replicate
# the unit cell in each direction. in this case, we replicate the unit cell
# once in the x direction, three times in the y direction, and twice in the z
# direction. it returns a new Net object, which we can assign to a new variable.
supercell_net = sod_net.replicate([1, 3, 2])

In [8]:
# we can now proceed as usual.
supercell_net.add_zif_atoms(template='H')
supercell_net.assign_mofff_topology()
supercell_net.to_lammps_data(eg_dir / 'ZIF-8-sod-cg-decorated-H-1x3x2.data')